In [78]:
#Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import math
import pycountry_convert as pc
from countries_dataset import countries


In [82]:
#Import excel files after initial row cleaning

#Production data in thousands 60kg bags
df_production = pd.read_excel('datasets/1a - Total production.xlsx',skiprows=3,skipfooter=3) 
df_production['Country'] = df_production['Country'].str.lstrip()
df_production['Continent'] = df_production['Country'].map(countries)

#Domestic Consumption data in thousands 60kg bags
df_consumption = pd.read_excel('datasets/1b - Domestic consumption.xlsx',skiprows=3,skipfooter=3) 
df_consumption['Country'] = df_consumption['Country'].str.lstrip()
df_consumption['Continent'] = df_consumption['Country'].map(countries)

#Gross Openings 
df_openings = pd.read_excel('datasets/1d - Gross Opening stocks.xlsx',skiprows=3,skipfooter=3) 
df_openings['Country'] = df_openings['Country'].str.lstrip()
df_openings['Continent'] = df_openings['Country'].map(countries)

#Exports in thousands 60kg bags
df_exports = pd.read_excel('datasets/1e - Exports - crop year.xlsx',skiprows=3,skipfooter=3) 
df_exports['Country'] = df_exports['Country'].str.lstrip()
df_exports['Continent'] = df_exports['Country'].map(countries)

#Imports in thousands 60kg bags
df_imports = pd.read_excel('datasets/2b - Imports.xlsx',skiprows=3,skipfooter=3) 
df_imports['Country'] = df_imports['Country'].str.lstrip()
df_imports['Continent'] = df_imports['Country'].map(countries)

d:\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
d:\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [85]:
df_production.columns

Index(['Country', 'Group', '1990/91', '1991/92', '1992/93', '1993/94',
       '1994/95', '1995/96', '1996/97', '1997/98', '1998/99', '1999/00',
       '2000/01', '2001/02', '2002/03', '2003/04', '2004/05', '2005/06',
       '2006/07', '2007/08', '2008/09', '2009/10', '2010/11', '2011/12',
       '2012/13', '2013/14', '2014/15', '2015/16', '2016/17', '2017/18',
       '2018/19', '2019/20', 'Totals', 'Continent'],
      dtype='object')

In [91]:
# sort the resulting DataFrame by production in descending order and select the top 10 countries
top_10_producing_countries = df_production[['Country','Totals']].sort_values(by='Totals', ascending=False).head(10)

# print the resulting DataFrame
top_10_producing_countries


,Country,Totals
2,Brazil,1.251382e+06
53,Viet Nam,4.800550e+05
23,Colombia,3.599796e+05
5,Indonesia,2.567497e+05
29,Ethiopia,1.444943e+05
36,India,1.370363e+05
41,Mexico,1.308928e+05
32,Guatemala,1.170068e+05
35,Honduras,1.112128e+05
51,Uganda,9.865552e+04
